# MUSIC RECOMMENDATION SYSTEM USING LSH MODEL 

# IMPORT THE IMPORTANT LIBRARIES NEEDED

In [1]:
import librosa
import soundfile as sp
import numpy as np
import pandas as pd
import sklearn
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import pickle
import librosa.display
import matplotlib.pyplot as plt
import time
from scipy.spatial.distance import cosine
from scipy.spatial.distance import euclidean
import os, glob

# READ EVERY FILE INSIDE THE AUDIO DIRECTORY AND TAKE MFCC AND DO PRE PROCESSING OF MFCC

In [2]:
import os, glob

path = 'Data/'
count = 0

files = glob.glob(path + '*.mp3')

P_dict = {}

for filename in tqdm(files):
    x, Sr = librosa.load(filename, dtype='float32')
    MFCC_1 = librosa.feature.mfcc(y=x,sr=Sr)
    
    Audios = []
    
    for index, value in enumerate(MFCC_1):
    
        length_of_vector = len(MFCC_1[index])

        Random_vector = np.random.rand(length_of_vector,1)

        Dot_product = np.dot(MFCC_1[index], Random_vector)

        Audios.append(float(Dot_product))
        
    for index, value in enumerate(Audios):
    
        if value > 1:
            Audios[index] = 1
        elif value < 1:
            Audios[index] = 0
    
    P_dict[filename] = Audios    

100%|██████████| 823/823 [01:25<00:00,  9.64it/s]


# TESTING IF CODE WORKS FOR ONE AUDIO FILE

In [28]:
path = 'Data/001039.mp3'

X, sr = librosa.load(path, dtype='float32')
MFCC = librosa.feature.mfcc(y=X, sr=sr)

In [29]:
Audio_1 = []

for index, value in enumerate(MFCC):
    
    length_of_vector = len(MFCC[index])
    
    Random_vector = np.random.rand(length_of_vector,1)
    
    Dot_product = np.dot(MFCC[index], Random_vector)
    
    Audio_1.append(Dot_product)

In [30]:
for index, value in enumerate(Audio_1):
    
    if value > 1:
        Audio_1[index] = 1
    elif value < 1:
        Audio_1[index] = 0
        
Audio_1

[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1]

# TAKING RANDOM PERMUTATIONS AND STORING INSIDE A LIST

In [3]:
P_lists = []

for i in range(20):
    Permutations = list(np.random.permutation(20))
    P_lists.append(Permutations)
    
P_lists

[[18, 1, 8, 19, 10, 11, 13, 3, 2, 5, 6, 15, 16, 7, 0, 4, 9, 12, 17, 14],
 [19, 1, 7, 0, 4, 12, 2, 9, 15, 5, 6, 16, 3, 10, 14, 8, 17, 13, 11, 18],
 [8, 2, 1, 0, 15, 10, 13, 7, 4, 6, 9, 19, 17, 16, 14, 18, 3, 11, 5, 12],
 [1, 13, 19, 16, 15, 6, 8, 10, 11, 17, 4, 2, 7, 3, 9, 0, 5, 18, 12, 14],
 [16, 7, 0, 1, 2, 10, 9, 14, 4, 12, 13, 18, 11, 5, 17, 8, 3, 6, 15, 19],
 [6, 18, 2, 3, 12, 4, 15, 14, 5, 19, 9, 10, 8, 1, 16, 7, 11, 13, 17, 0],
 [1, 8, 11, 6, 7, 16, 18, 13, 12, 3, 15, 5, 4, 19, 17, 9, 2, 14, 10, 0],
 [5, 8, 18, 17, 15, 10, 0, 16, 14, 13, 11, 7, 12, 6, 2, 1, 4, 9, 19, 3],
 [15, 9, 11, 8, 14, 18, 12, 16, 5, 0, 17, 7, 3, 6, 19, 1, 2, 13, 10, 4],
 [19, 7, 10, 8, 15, 17, 12, 4, 16, 14, 3, 9, 18, 11, 6, 1, 13, 0, 2, 5],
 [17, 16, 14, 11, 1, 3, 0, 19, 12, 2, 4, 15, 5, 7, 8, 6, 18, 13, 9, 10],
 [2, 0, 18, 4, 8, 12, 3, 5, 1, 16, 17, 11, 10, 15, 6, 7, 9, 13, 14, 19],
 [1, 9, 15, 17, 19, 2, 8, 3, 11, 7, 10, 4, 0, 14, 12, 18, 6, 13, 16, 5],
 [11, 10, 19, 17, 7, 5, 14, 0, 18, 15, 16, 6, 2, 8,

# STORE THE WHOLE LISTS OF PERMUTATIONS INSIDE A PICKLE FILE

In [4]:
name = "Permutations.pkl"

with open(name, 'wb') as file:  
    pickle.dump(P_lists, file)

In [10]:
with open(r"Permutations.pkl", 'rb') as file: 
    list_of_perm = pickle.load(file)

In [249]:
list_of_perm

[[8, 16, 12, 9, 15, 0, 4, 17, 13, 11, 18, 10, 1, 6, 5, 14, 19, 2, 7, 3],
 [6, 11, 18, 19, 8, 17, 3, 2, 0, 14, 1, 5, 13, 12, 7, 4, 9, 16, 10, 15],
 [6, 11, 3, 5, 10, 12, 17, 4, 15, 13, 7, 18, 19, 0, 16, 14, 2, 9, 8, 1],
 [17, 5, 14, 0, 15, 18, 19, 9, 8, 12, 10, 6, 7, 11, 2, 13, 3, 4, 16, 1],
 [6, 16, 5, 11, 10, 18, 7, 12, 9, 4, 19, 1, 0, 15, 3, 13, 2, 8, 14, 17],
 [2, 19, 17, 13, 18, 10, 12, 7, 5, 14, 15, 1, 6, 9, 16, 11, 3, 8, 0, 4],
 [19, 6, 15, 10, 7, 14, 17, 11, 13, 12, 3, 16, 5, 9, 8, 1, 18, 0, 4, 2],
 [14, 7, 12, 5, 8, 1, 16, 11, 6, 15, 17, 3, 9, 18, 13, 19, 4, 2, 0, 10],
 [10, 18, 3, 12, 5, 11, 0, 1, 15, 14, 6, 16, 4, 19, 9, 13, 7, 17, 8, 2],
 [15, 9, 19, 5, 18, 16, 14, 8, 17, 3, 6, 4, 2, 12, 11, 1, 10, 0, 7, 13],
 [15, 8, 9, 14, 4, 17, 18, 6, 7, 2, 19, 11, 12, 0, 3, 5, 16, 13, 1, 10],
 [6, 12, 15, 1, 3, 11, 8, 2, 16, 0, 4, 14, 7, 13, 18, 19, 10, 5, 17, 9],
 [1, 9, 10, 11, 17, 7, 8, 6, 12, 15, 14, 3, 16, 4, 18, 5, 13, 19, 0, 2],
 [14, 9, 10, 17, 19, 6, 13, 4, 12, 5, 15, 7, 1, 8, 

# TESTING THE PERMUTATIONS AND MAKING BUCKETS

In [31]:
newlist = []
anotherList = []

for i in range(len(list_of_perm)):
    
    anotherList = []
    
    for j in range(len(Audio_1)):

        if Audio_1[j] == 1:
            anotherList.append(list_of_perm[i][j])
    
    newlist.append(min(anotherList))
    
newlist

[1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]

In [32]:
len(newlist)

20

In [33]:
newlist

[1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]

In [34]:
Audio_1

[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1]

# STORE THE DICTIONARY OF MFCC PRE PROCESSING INSIDE A PICKLE FILE

In [5]:
name = "Audios_shingling.pkl"

with open(name, 'wb') as file:  
    pickle.dump(P_dict, file)

In [6]:
with open(r"Audios_shingling.pkl", 'rb') as file: 
    Audios = pickle.load(file)

# MAKE A DATAFRAME OF THE WHOLE DICTIONARIES

In [7]:
df = pd.DataFrame(Audios)
df

,Data\001039.mp3,Data\001040.mp3,Data\001066.mp3,Data\001069.mp3,Data\001073.mp3,Data\001075.mp3,Data\001082.mp3,Data\001083.mp3,Data\001087.mp3,Data\001102.mp3,...,Data\020373.mp3,Data\020374.mp3,Data\020375.mp3,Data\020376.mp3,Data\020424.mp3,Data\020432.mp3,Data\020469.mp3,Data\020667.mp3,Data\020704.mp3,Data\020818.mp3
0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,1,1,1,1,1,1
3,1,1,1,1,1,1,0,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,1,1,1,0,0,1,0,0,0,1,...,0,0,0,0,1,1,1,1,1,1
5,1,1,1,1,1,1,0,1,1,1,...,1,0,1,1,1,1,1,1,1,1
6,1,0,0,0,1,1,0,1,1,1,...,0,0,0,0,1,0,0,1,0,1
7,1,1,1,1,1,1,0,1,1,1,...,1,0,0,0,0,1,0,1,1,1
8,1,0,0,1,0,1,0,1,1,1,...,0,0,0,0,1,0,0,0,0,1
9,1,1,1,1,1,1,0,1,1,1,...,0,1,0,0,1,0,0,1,0,0


# GET COLUMN NAMES FROM DATAFRAME

In [8]:
Columns = list(df.columns)

# NOW GET PERMUTATIONS AND HAVE MINIMUM VALUES FOR EACH AUDIO FILE NOW 

In [11]:
newlist = []
anotherList = []
MinHash_dict = {}

for c in range(len(Columns)):
    
    newlist = []

    for i in range(len(list_of_perm)):

        anotherList = []

        for j in range(len(df[Columns[c]])):

            if df[Columns[c]][j] == 1:
                anotherList.append(list_of_perm[i][j])
        
        newlist.append(min(anotherList))
    
    MinHash_dict[Columns[c]] = newlist    

# THE MIN HASH DICTIONARY

In [12]:
MinHash_dict[Columns[100]]

[1, 0, 0, 6, 0, 2, 6, 8, 8, 7, 3, 0, 2, 5, 2, 4, 4, 1, 0, 0]

# PICKLE THE MINHASH_DICT 

In [13]:
name = "MinHash_dict.pkl"

with open(name, 'wb') as file:  
    pickle.dump(MinHash_dict, file)

In [14]:
with open(r"MinHash_dict.pkl", 'rb') as file: 
    Minimum = pickle.load(file)

In [15]:
Minimum[Columns[100]]

[1, 0, 0, 6, 0, 2, 6, 8, 8, 7, 3, 0, 2, 5, 2, 4, 4, 1, 0, 0]

# BANDS OF EACH AUDIO FILE TO THEN COMPARE WITH THE AUDIO FILE WE UPLOAD

In [16]:
Band_1 = {}
Band_2 = {}
sum1 = 0
sum2 = 0

for i in Minimum:
    
    sum1 = 0
    sum2 = 0
    
    for j in range(20):
        
        if j < 10:
            
            sum1 += Minimum[i][j]
        
        if j > 10:
            
            sum2 += Minimum[i][j]
            
    Band_1[i] = sum1
    Band_2[i] = sum2

# GET BOTH SIMILAR SUMS IN ONE DICTIONARY

In [17]:
Final_band_1 = {}
Final_band_2 = {}

for key, value in Band_1.items():
    if value not in Final_band_1:
        Final_band_1[value] = [key]
    else:
        Final_band_1[value].append(key)

for key, value in Band_2.items():
    if value not in Final_band_2:
        Final_band_2[value] = [key]
    else:
        Final_band_2[value].append(key)

In [18]:
len(Final_band_1[6])

35

In [19]:
len(Final_band_2[4])

46

In [20]:
len(Final_band_1)

46

In [21]:
len(Final_band_2)

39

# DUMP THE FINAL BANDS INTO A PICKLE FILE AS WE NEED THEM LATER

In [22]:
name = "Final_Bands_1.pkl"

with open(name, 'wb') as file:  
    pickle.dump(Final_band_1, file)

In [23]:
name = "Final_Bands_2.pkl"

with open(name, 'wb') as file:  
    pickle.dump(Final_band_2, file)

# LOAD THE FINAL BANDS FROM PICKLE FILES

In [24]:
Final_band_1 = {}
Final_band_2 = {}

with open(r"Final_bands_1.pkl", 'rb') as file: 
    Final_band_1 = pickle.load(file)
    
with open(r"Final_bands_2.pkl", 'rb') as file: 
    Final_band_2 = pickle.load(file)

In [25]:
len(Final_band_1)

46

In [26]:
len(Final_band_2)

39

# TESTING AND GETTING JACCARD 

In [35]:
newlist = []
anotherList = []

for i in range(len(list_of_perm)):
    
    anotherList = []
    
    for j in range(len(Audio_1)):

        if Audio_1[j] == 1:
            anotherList.append(list_of_perm[i][j])
    
    newlist.append(min(anotherList))
    
newlist

[1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]

In [36]:
Sum1 = 0
Sum2 = 0
test_Audio = {}

for i in range(20):
    if i < 10:
        Sum1 += newlist[i]
        
    if i > 10:
        Sum2 += newlist[i]
    
    
test_Audio['sum_1'] = Sum1
test_Audio['sum_2'] = Sum2

In [37]:
test_Audio

{'sum_1': 2, 'sum_2': 1}

In [38]:
Similarity_1 = []
Similarity_2 = []

for i in Final_band_1:
    if test_Audio['sum_1'] == i:
        Similarity_1 = Final_band_1[i]
        
for i in Final_band_2:
    if test_Audio['sum_2'] == i:
        Similarity_2 = Final_band_2[i]

In [39]:
len(Similarity_1)

49

In [40]:
len(Final_band_2[4])

46

In [41]:
Similar = Similarity_1 + Similarity_2

In [42]:
Similar = list(np.unique(Similar))
len(Similar)

73

In [43]:
Hash_sum = 0
Jaccard = {}

for i in Similar:
    
    Hash_sum = 0
    
    for index, value in enumerate(newlist):
        
        if Minimum[i][index] == newlist[index]:
            Hash_sum += 1
    
    Jaccard[i] = (Hash_sum / 20) * 100

# PRINT THE LOWEST / HIGHEST METRIC VALUE AUDIO 

In [44]:
i = 0

for index, value in enumerate(Jaccard):
    if Jaccard[value] == Jaccard[max(Jaccard, key=Jaccard.get)]:
        print('THE HIGHEST METRIC VALUE AUDIO IS ', value[5:], ' WITH JACCARD = ', Jaccard[max(Jaccard, key=Jaccard.get)])
        break
i = 0 

for index, value in enumerate(Jaccard):
    if Jaccard[value] == Jaccard[min(Jaccard, key=Jaccard.get)]:
        print('THE LOWEST METRIC VALUE AUDIO IS ', value[5:], ' WITH JACCARD = ', Jaccard[min(Jaccard, key=Jaccard.get)])
        break
        

THE HIGHEST METRIC VALUE AUDIO IS  001039.mp3  WITH JACCARD =  100.0
THE LOWEST METRIC VALUE AUDIO IS  007378.mp3  WITH JACCARD =  45.0


# Here we can see that our model predicted accurately that it was which audio file it also gave us the least similar if we show some of the similar audio files then we have our recommender system ready and running

# COMBINE CODE OF LSH MODEL 

In [154]:
def Songs_Entry():
    path = 'Data/'
    count = 0

    files = glob.glob(path + '*.mp3')

    P_dict = {}

    for filename in tqdm(files):
        x, Sr = librosa.load(filename, dtype='float32')
        MFCC_1 = librosa.feature.mfcc(y=x,sr=Sr)

        Audios = []

        for index, value in enumerate(MFCC_1):

            length_of_vector = len(MFCC_1[index])

            Random_vector = np.random.rand(length_of_vector,1)

            Dot_product = np.dot(MFCC_1[index], Random_vector)

            Audios.append(float(Dot_product))

        for index, value in enumerate(Audios):

            if value > 1:
                Audios[index] = 1
            elif value < 1:
                Audios[index] = 0

        P_dict[filename] = Audios  
    
    return P_dict

In [155]:
def permutations(number_of_permutations):
    P_lists = []

    for i in range(number_of_permutations):
        Permutations = list(np.random.permutation(20))
        P_lists.append(Permutations)
    
    return P_lists

In [156]:
def MinHashDict(Columns, list_of_perm, df_):
    newlist = []
    anotherList = []
    MinHash_dict = {}

    for c in range(len(Columns)):

        newlist = []

        for i in range(len(list_of_perm)):

            anotherList = []

            for j in range(len(df_[Columns[c]])):

                if df_[Columns[c]][j] == 1:
                    anotherList.append(list_of_perm[i][j])

            newlist.append(min(anotherList))

        MinHash_dict[Columns[c]] = newlist
    
    return MinHash_dict

In [157]:
def FinalBands(minimum):
    Band_1 = {}
    Band_2 = {}
    sum1 = 0
    sum2 = 0

    for i in minimum:

        sum1 = 0
        sum2 = 0

        for j in range(30):

            if j < 15:

                sum1 += minimum[i][j]

            if j > 15:

                sum2 += minimum[i][j]

        Band_1[i] = sum1
        Band_2[i] = sum2
            
    
    Final_band_1 = {}
    Final_band_2 = {}

    for key, value in Band_1.items():
        if value not in Final_band_1:
            Final_band_1[value] = [key]
        else:
            Final_band_1[value].append(key)

    for key, value in Band_2.items():
        if value not in Final_band_2:
            Final_band_2[value] = [key]
        else:
            Final_band_2[value].append(key)
            
    return Final_band_1, Final_band_2

In [158]:
def PreProcessQuery(query_):
    path =  query_ + '.mp3'
        
    X, sr = librosa.load(path, dtype='float32')
    MFCC = librosa.feature.mfcc(y=X, sr=sr)

    query = []

    for index, value in enumerate(MFCC):

        length_of_vector = len(MFCC[index])

        Random_vector = np.random.rand(length_of_vector,1)

        Dot_product = np.dot(MFCC[index], Random_vector)

        query.append(Dot_product)

    for index, value in enumerate(query):

        if value > 1:
            query[index] = 1
        elif value < 1:
            query[index] = 0
            
    return query

In [159]:
def LSHModel(query_audio):
    with open(r"Audios_shingling.pkl", 'rb') as file: 
        Audios = pickle.load(file)
        
    Audio_query = PreProcessQuery(query_audio)
        
    Dict_of_hashes = Audios
    
    list_of_perm = permutations(30)
    
    df = pd.DataFrame(Dict_of_hashes)
    
    Columns = list(df.columns)
    
    Minimum = MinHashDict(Columns, list_of_perm, df) 
        
    Final_band_1, Final_band_2 = FinalBands(Minimum)
    
    newlist = []
    anotherList = []

    for i in range(len(list_of_perm)):

        anotherList = []

        for j in range(len(Audio_query)):

            if Audio_query[j] == 1:
                anotherList.append(list_of_perm[i][j])

        newlist.append(min(anotherList))

    Sum1 = 0
    Sum2 = 0
    test_Audio = {}

    for i in range(30):
        if i < 15:
            Sum1 += newlist[i]

        if i > 15:
            Sum2 += newlist[i]


    test_Audio['sum_1'] = Sum1
    test_Audio['sum_2'] = Sum2
    
    Similarity_1 = []
    Similarity_2 = []

    for i in Final_band_1:
        if test_Audio['sum_1'] == i:
            Similarity_1 = Final_band_1[i]

    for i in Final_band_2:
        if test_Audio['sum_2'] == i:
            Similarity_2 = Final_band_2[i]
            
    Similar = Similarity_1 + Similarity_2
    Similar = list(np.unique(Similar))
    
    Hash_sum = 0
    Jaccard = {}

    for i in Similar:

        Hash_sum = 0

        for index, value in enumerate(newlist):

            if Minimum[i][index] == newlist[index]:
                Hash_sum += 1

        Jaccard[i] = (Hash_sum / 30)*100
        
    i = 0

    for index, value in enumerate(Jaccard):
        if Jaccard[value] == Jaccard[max(Jaccard, key=Jaccard.get)]:
            print('THE HIGHEST METRIC VALUE AUDIO IS ', value[5:], ' WITH JACCARD = ', Jaccard[max(Jaccard, key=Jaccard.get)])
            pre = value[5:]
            pre = pre[:6]
            return pre
            break
    '''i = 0 

    for index, value in enumerate(Jaccard):
        if Jaccard[value] == Jaccard[min(Jaccard, key=Jaccard.get)]:
            print('THE LOWEST METRIC VALUE AUDIO IS ', value[5:], ' WITH JACCARD = ', Jaccard[min(Jaccard, key=Jaccard.get)])
            break
            
    '''

# THE QUERY AUDIO AND PUT INSIDE LSH MODEL TO PREDICT 

In [242]:
import time

start_time = time.time()

print(LSHModel('Data/012058'))

end_time = time.time()

elapsed_time = end_time - start_time

print("Elapsed time: {:.2f} seconds".format(elapsed_time))

THE HIGHEST METRIC VALUE AUDIO IS  012058.mp3  WITH JACCARD =  100.0
012058
Elapsed time: 2.15 seconds


# EVALUATE LSH

In [ ]:
def getMFCCs():

    path = 'Data/'
    count = 0

    files = glob.glob(path + '*.mp3')

    MFCC_dict = {}

    for filename in tqdm(files):
        x, Sr = librosa.load(filename, dtype='float32')
        MFCC_1 = librosa.feature.mfcc(y=x,sr=Sr)

        filename = filename[5:]
        MFCC_dict[filename[:6]] = MFCC_1 
        
    return MFCC_dict

In [157]:
name = "Audios_MFCCs.pkl"

with open(name, 'wb') as file:  
    pickle.dump(queries, file)

In [198]:
path = 'Data/'

files = glob.glob(path + '*.mp3')
filenames = []
number_correct = 0

for file in files:
    file = file[5:]
    file = file[:6]
    filenames.append(file)

n = 5
    
for audio in tqdm(filenames):
    
    for _ in range(n):
        predict = LSHModel(audio)
    
    audio_dot = PreProcessQuery(audio)
    predict_dot = PreProcessQuery(predict)
    
    if audio_dot == predict_dot:
        number_correct += 1
    
print('Accuracy : ', (number_correct / len(filenames)) * 100)

100%|██████████████████████████████████████████████████████████████████████████████| 823/823 [2:21:09<00:00, 10.29s/it]

Accuracy :  48.48116646415553


## If we evaluate as I havent had the most iterations but with only 5 iterations and training my dataset we got around 49 percent accuracy which I calculated myself but if we compare to KNN LSH works far more better in similar audio matchig, I dont know the genres but I have tested them out and my LSH model does give similar audios if not same like the query 

# ---------------------------------------------------------------------------------------------------------------------------

## The above accuracy is the accuracy which shows how much my LSH model can predict 100% same audio file but it predicts similar audio file so the accuracy here is fine, I checked for all the audio files and my LSH model gives accurate similar audio.

## GET THE NAME, FILE PATH AND META DATA AND SAVE INTO MONGODB NOSQL DATABASE

In [2]:
import os
import pymongo
from mutagen.easyid3 import EasyID3

## Establish a connection to MongoDB instance

In [3]:
client = pymongo.MongoClient("mongodb://localhost:27017")
db = client["Music_Database"]
collection = db["All_mp3"]

## Define a function to extract the metadata from a music file

In [4]:
def get_metadata(file_path):
    metadata = EasyID3(file_path)
    metaData = {"title": metadata.get("title", [""])[0],
        "artist": metadata.get("artist", [""])[0],
        "album": metadata.get("album", [""])[0],
        "genre": metadata.get("genre", [""])[0]
        }
    return metaData;

## Loop over music files and insert them into MongoDB

In [5]:
for root, dirs, files in os.walk("C:/Users/ahaqu/OneDrive/Desktop/ML BASED SE PROJECT/Data"):
    for file_name in files:
        if file_name.endswith(".mp3"):
            file_path = os.path.join(root, file_name)
            metadata = get_metadata(file_path)
            with open(file_path, "rb") as f:
                file_data = f.read()
            collection.insert_one({
                "file_path": file_path,
                "file_name": file_name,
                **metadata, ## HERE I USED ** TO UNPACK DICTIONARY OF META DATA INTO KEY VALUE PAIRS
                "data": file_data
            })

Here all my meta data and also the information I need is saved into mongodb 